In [378]:
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import scipy as sci
import autograd as ad
import autograd.numpy as np
from cosmatrix import cosm,expm_pade
expm = expm_pade
from autograd.extend import primitive, defvjp
from qutip import *
def annihilation(dim):
    return np.diag(np.sqrt(np.arange(1,dim)),1)
def creation(dim):
    return np.diag(np.sqrt(np.arange(1,dim)),-1)
def SNAIL_H(phi_ex,beta,N,Ej,Ec):
    phi_ex = 2*np.pi*phi_ex
    @primitive
    def Us_min(phi_ex):
        def U_s(phi): 
            return (-beta*np.cos(phi-phi_ex)-N*np.cos((phi)/N))
        phi_min = sci.optimize.minimize(U_s,0).x
        return phi_min
    
    def phi_minde(ans, phi_ex):
        def phi_minde_vjp(g):
            c2 = beta*np.cos(ans - phi_ex) + 1/N*np.cos(ans/N)
            return g*beta*np.cos(ans - phi_ex)/c2
        return phi_minde_vjp
    defvjp(Us_min, phi_minde)
    
    phi_min = Us_min(phi_ex)
    # potential expansion around minimum
    c2 = beta*np.cos(phi_min - phi_ex) + 1/N*np.cos(phi_min/N)
    omega_s = np.sqrt(8*c2*Ej*Ec)
    phi_zpf = np.power(2*Ec/(Ej*c2),1/4)
    g2 = Ej*phi_zpf**2*c2/2
    sdim = 10
    s = annihilation(sdim)
    sd = creation(sdim)
    x2 = np.matmul(s+sd,s+sd)
    c3 = (N**2-1)/N**2*np.sin(phi_min/N)
    g3 = Ej*phi_zpf**3*c3/3/2
    c4 = -beta*np.cos(phi_min-phi_ex) -1/N**3*np.cos(phi_min/N)

    g4 = Ej*phi_zpf**4*c4/4/3/2
    Hs = (omega_s * np.matmul(sd,s)
        - Ej*(beta*cosm(phi_zpf*(s+sd)+(phi_min-phi_ex)*np.identity(sdim))
        + N*cosm((phi_zpf*(s+sd)+phi_min*np.identity(sdim))/N))- g2*x2)
    energy,U = np.linalg.eigh(Hs)
    U_dag = np.conjugate(U.transpose())
    Hs = U_dag@Hs@U
    return Hs-energy[0]*np.identity(sdim),U_dag@(s+sd)@U

def coupled_spectrum(phi_ex,omega_c,g_intfa):
    beta = 0.15
    N = 3
    phi_ex = 2*np.pi*phi_ex
    @primitive
    def Us_min(phi_ex):
        def U_s(phi): 
            return (-beta*np.cos(phi-phi_ex)-N*np.cos((phi)/N))
        phi_min = sci.optimize.minimize(U_s,0).x
        return phi_min
    
    def phi_minde(ans, phi_ex):
        def phi_minde_vjp(g):
            c2 = beta*np.cos(ans - phi_ex) + 1/N*np.cos(ans/N)
            return g*beta*np.cos(ans - phi_ex)/c2
        return phi_minde_vjp
    defvjp(Us_min, phi_minde)
    
    phi_min = Us_min(phi_ex)
    # potential expansion around minimum
    c2 = beta*np.cos(phi_min - phi_ex) + 1/N*np.cos(phi_min/N)
    omega_s = np.sqrt(8*c2*Ej*Ec)
    phi_zpf = np.power(2*Ec/(Ej*c2),1/4)
    g2 = Ej*phi_zpf**2*c2/2
    sdim = 10
    s = annihilation(sdim)
    sd = creation(sdim)
    x2 = np.matmul(s+sd,s+sd)
    Hs = (omega_s * np.matmul(sd,s)
        - Ej*(beta*cosm(phi_zpf*(s+sd)+(phi_min-phi_ex)*np.identity(sdim))
        + N*cosm((phi_zpf*(s+sd)+phi_min*np.identity(sdim))/N))- g2*x2)
    cdim = 5
    c = annihilation(cdim)
    cd = creation(cdim)
    Hc = omega_c*np.matmul(cd,c)
    Ic = np.identity(cdim)
    Is = np.identity(sdim)
    Hs = np.kron(Hs,Ic)
    Hc = np.kron(Is,Hc)
    g_int = g_intfa*2*np.pi*np.power((2*Ec)/Ej,1/4)/phi_zpf
    H_int = g_int*np.kron(s+sd,cd+c)
#     H_int = g_int*(np.kron(sd,c)+np.kron(s,cd))
    H = Hs + Hc + H_int
    energy,dressed_states = np.linalg.eigh(H)
    energy,U = np.linalg.eigh(H)
    U_dag = np.conjugate(U.transpose())
    H = U_dag@H@U

    return H-energy[0]*np.identity(sdim*cdim),U_dag@np.kron(s+sd,Ic)@U

In [379]:
def find_optimal_k(A, B, D):
    # Define a large initial minimum difference
    min_diff = float('inf')
    optimal_k = None
    
    # Iterate over a range of possible k values
    # The range can be adjusted based on expected size of k or other insights you have about your problem
    for k in range(-1000, 1000):
        # Calculate the difference for this value of k
        diff = abs(A - (B + k * D))
        
        # If this is the smallest difference we've found so far, update min_diff and optimal_k
        if diff < min_diff:
            min_diff = diff
            optimal_k = k
            
    return optimal_k
# Function to calculate overlap (you might use inner product, fidelity, etc.)
def calculate_overlap(state1, state2):
    return abs((state1.dag() * state2)[0,0])**2

In [380]:
import qutip as qt
def floquet_spec(omega,A,phi_ex):
#     phi_ex = 0.41340478058712055
    H0,Hc = SNAIL_H(phi_ex,beta,N,Ej,Ec)
    energies,U=np.linalg.eig(H0)
    energies = energies - energies[0]*np.ones(len(energies))
    energies = energies/2/np.pi
    # U_dag = np.conjugate(U.transpose())
    # U_dag@Hc@U
    energies = np.sort(energies)
    omega_s = energies[1]
    H0 = Qobj(H0)
    Hc = Qobj(Hc)
    args = {'w': omega}
    T = (2*np.pi)/omega
    H = [H0, [Hc, lambda t, args: A*np.cos(args['w']*t)]] 
    options = Options(nsteps=100000)  # Increasing nsteps to 10000, adjust as needed
    f_modes, f_energies = floquet_modes(H, T, args, False, options=options) 
    
    # Define your system dimension
    sdim = 10
    floquet_states = f_modes
    # Create your basis states
    basis_states = [qt.basis(sdim, i) for i in range(3)]
    # Initialize a list to hold the indices of the Floquet states with the maximum overlap for each basis state
    max_overlap_indices = [-1] * 3
    max_overlaps = [0] * 3
    # Loop over each Floquet state
    for f_index, f_state in enumerate(floquet_states):

        # Loop over each basis state
        for b_index, b_state in enumerate(basis_states):

            # Calculate the overlap
            overlap = calculate_overlap(f_state, b_state)

            # If this is the maximum overlap for this basis state, store the index
            if overlap > max_overlaps[b_index]:
                max_overlaps[b_index] = overlap
                max_overlap_indices[b_index] = f_index
    energy01 = f_energies[max_overlap_indices[1]]/2/np.pi-f_energies[max_overlap_indices[0]]/2/np.pi
    k = find_optimal_k(omega_s, energy01, omega/2/np.pi)
    energy01 =energy01+k*omega/2/np.pi
    omega_ss = energy01-omega_s
    energy02 = f_energies[max_overlap_indices[2]]/2/np.pi-f_energies[max_overlap_indices[0]]/2/np.pi
    k = find_optimal_k(2*omega_s, energy02, omega/2/np.pi)
    energy02 =energy02+k*omega/2/np.pi
    anh = energy02 - energy01 - energy01
    if (max_overlaps<(np.sqrt(2)/2*np.ones(3))).all():
        omega_ss = 1000
        anh = 1000
    return np.abs(omega_ss),anh
def floquet_comp(omega,A,phi_ex,omega_c):
    omega_c = omega_c *2*np.pi
    g_intfa = 0.15
    N = 3
    H0,Hc = coupled_spectrum(phi_ex,omega_c,g_intfa)
    energies,U=np.linalg.eig(H0)
    energies = energies - energies[0]*np.ones(len(energies))
    energies = energies
    # U_dag = np.conjugate(U.transpose())
    # U_dag@Hc@U
    energies = np.sort(energies)
    omega_cp = energies[1]
    H0 = Qobj(H0)
    Hc = Qobj(Hc)
    args = {'w': omega}
    T = (2*np.pi)/omega
    H = [H0, [Hc, lambda t, args: A*np.cos(args['w']*t)]] 
    options = Options(nsteps=100000)  # Increasing nsteps to 10000, adjust as needed
    f_modes, f_energies = floquet_modes(H, T, args, False, options=options) 
    
    # Define your system dimension
    sdim = 10
    cdim = 5
    floquet_states = f_modes
    # Create your basis states
    basis_states = [qt.basis(sdim*cdim, i) for i in range(2)]
    # Initialize a list to hold the indices of the Floquet states with the maximum overlap for each basis state
    max_overlap_indices = [-1] * 3
    max_overlaps = [0] * 3
    # Loop over each Floquet state
    for f_index, f_state in enumerate(floquet_states):

        # Loop over each basis state
        for b_index, b_state in enumerate(basis_states):

            # Calculate the overlap
            overlap = calculate_overlap(f_state, b_state)

            # If this is the maximum overlap for this basis state, store the index
            if overlap > max_overlaps[b_index]:
                max_overlaps[b_index] = overlap
                max_overlap_indices[b_index] = f_index
    energy01 = f_energies[max_overlap_indices[1]]-f_energies[max_overlap_indices[0]]
    k = find_optimal_k(omega_cp, energy01, omega)
    energy01 =energy01+k*omega
    omega_cpp = energy01
    if (max_overlaps<(np.sqrt(2)/2*np.ones(3))).all():
        omega_cpp == 2*np.pi*1000
    return omega_cpp
def energy_der(omega,A,phi_ex,omega_c):
    delta = 1e-6
    der = (-floquet_comp(omega,A,phi_ex,omega_c) + floquet_comp(omega,A,phi_ex+delta,omega_c))/delta
    return der
def T1f(der):
    factor = np.sqrt(2*np.abs(np.log(2*np.pi*1*1e-5)))
    return 1/(factor*10**-6*der)
import scipy.optimize as opt
def minimize_anh( omega,A,phi_ex):
    # Wrapper function that takes only phi_ex as an argument
    def anh_wrapper(phi_ex):
        omega_s, anh = floquet_spec(omega,A,phi_ex)
        return np.abs(anh)
    bounds = [(0,0.5)]
    # Optimize phi_ex
    result = opt.minimize(anh_wrapper,phi_ex, bounds = bounds,method='Nelder-Mead')

    # Return the result of the optimization
    return result.x, result.fun
def minimize_Tp1(omega,A,phi_ex,omega_c):
    # Wrapper function that takes only phi_ex as an argument
    def wrapper(omega_c):
        der = energy_der(omega,A,phi_ex,omega_c)
        T1fs = np.abs(T1f(der))
        return 1/T1fs*1e9

    # Optimize phi_ex
    bounds = [(1,3)]
    result = opt.minimize(wrapper,omega_c, bounds = bounds,method='Nelder-Mead')

    # Return the result of the optimization
    return result.x, result.fun

In [381]:
beta = 0.12
N = 3
Ej = 15*2*np.pi
Ec = 0.7*2*np.pi
omega = 2*2*np.pi
omega_c = 2.5
A = 0*2*np.pi
phi_ex =0.38059900660029355
omega_s, anh = floquet_spec(omega,A,phi_ex)
np.abs(T1f(energy_der(omega,A,phi_ex,omega_c)))

38057870.48658439

In [382]:
energy_der(omega,A,phi_ex,omega_c)/2/np.pi

0.0009506793856442843

In [383]:
floquet_comp(omega,0,phi_ex,omega_c)/2/np.pi

2.4928995296843834

In [330]:

phi_ex,anh=minimize_anh(omega,A,0.41340478058712055)
phi_ex=phi_ex[0]


In [331]:
phi_ex,anh

(0.38433725695208854, 4.4149336922316706e-05)

In [332]:
omega_c, T=minimize_Tp1(omega,A,phi_ex,omega_c)

In [333]:
np.abs(T1f(energy_der(omega,A,phi_ex,omega_c)))

15996988389705.096

In [334]:
omega_s=np.abs(SNAIL_H(phi_ex,beta,N,Ej,Ec)[0][1][1]/2/np.pi)

In [335]:
omega_s

4.655954919953192

In [336]:
omega_c,T

(array([2.61740727]), 6.251176631743713e-05)

In [337]:
eta = A*2*omega_s/(omega_s**2-(omega/2/np.pi)**2)/2/np.pi

In [338]:
omega_cp=np.sqrt(2*(1+eta**2)*omega_s*Ec/2/np.pi)
np.abs((omega_cp-omega_c)/omega_c)

array([0.02456778])

In [339]:
omega_cp

2.553103383714508

In [340]:
omega_c

array([2.61740727])

In [348]:
(np.sqrt(2*7.75*0.3)-0.87)/0.87

1.4786044428560714